# Functional Closures and Decorators

##  Closures
* In order to understand closures, let's be sure we understand Python's scoping rules: LEGB
  * L = local
  * E = enclosing
  * G = global
  * B = builtin (e.g., len() function)

In [ ]:
global_var = 'global var'
   
def outer_func():
    enclosing_var = 'local to outer_func()'
    
    def inner_func():
        local_var = 'local to inner_func()'
        print(local_var)
        print(enclosing_var, 'enclosing scope')
        print(global_var, 'global scope')
        
    inner_func()

In [ ]:
outer_func()

* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function that remembers a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [ ]:
def make_adder(x):
    # print x's address
    print(f'id(x): 0x{id(x):x}') 
    # this is enclosing scope
    
    def adder(y):
        print('in adder')
        return x + y # Python uses LEGB to find 'x'
    
    print(f'id(adder): 0x{id(adder):x}')
    return adder # we return the function adder (i.e., its address in memory) 

add1 = make_adder(1)
print('about to call add1')
add1(5)

In [ ]:
# let's use repr so we can see the address of the function
print(type(add1), repr(add1), sep='\n')

In [ ]:
# all functions have a closure attribute
add1.__closure__

In [ ]:
# notice that the cell object has a reference to an int object
add1.__closure__[0].cell_contents

In [ ]:
print(make_adder.__closure__)

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [ ]:
def logging(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func}({args}, {kwargs})')
        return func(*args, **kwargs)
        
    return wrapper

In [ ]:
logging_add1 = logging(add1)

In [ ]:
print(add1(5))

In [ ]:
print(logging_add1(5))

In [ ]:
logging_add1.__closure__[0].cell_contents

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (__`*args`__)
* variable keyword args (__`**kwargs`__)
* functions are objects (actually everything in Python is an object)

In [ ]:
#from functools import wraps

def document_it(func):
    # below is a nested, or inner function
    # the @wraps decorator is needed so metadata (like .__doc__) is not lost
    #@wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

In [ ]:
def add_things(a, b):
    """Add 2 numbers."""
    return a + b

print('Running plain old add_things()')
print(add_things(13, 5))

In [ ]:
# manual decorator assignment
cooler_add_things = document_it(add_things) 

In [ ]:
#print('Running cooler_add_things()')
cooler_add_things(13, 5)

In [ ]:
# decorator shorthand for what we did above

@document_it
def add_things(a, b):
    """Add two numbers, a and b."""
    return a + b

# add_things = document_it(add_things)

print(add_things(13, -5))

## Lab: Decorators
1. Make a timer decorator that computes the elapsed time of the function wrapped by it
2. Create some function which takes an integer as its parameter
   * create a wrapper that ensures the parameter is positive
   * use that wrapper to decorate your original function
2. Make a "call counter" decorator that keeps track of how many times a function has been called
2. Make a "cache" decorator that stores function results in a dictionary so repeated calls with the same arguments return instantly rather than re-computing a result that has been previously computed